# Anlysis of museum collection

### Import all the module required

In [1]:
import pandas as pd
import numpy as np
import re

### Import the data from the website

In [2]:
collection_df = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv")
collection_df

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,16,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
0,1979.486.1,False,False,1,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
1,1980.264.5,False,False,2,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
2,67.265.9,False,False,3,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
3,67.265.10,False,False,4,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
4,67.265.11,False,False,5,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
5,67.265.12,False,False,6,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/6,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
6,67.265.13,False,False,7,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/7,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",Birds|Coins
7,67.265.14,False,False,8,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/8,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",Inscriptions|Men|Profiles|Coins
8,67.265.15,False,False,9,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/9,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN
9,1979.486.3,False,False,10,American Decorative Arts,Coin,Two-and-a-half-dollar Indian Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/10,3/4/2019 8:00:00 AM,"Metropolitan Museum of Art, New York, NY",NaN


## Data clean

The data now looks fine. We intend to do some data clean work as follows:
+ Check "Object Begin Date" and "Object End Date" with "Object Date"
+ Check "Artist Begin Date" and "Artist End Date" with "Artist Display Bio"
+ Check "Artist Display Name" with "Artist Alpha Sort", then delete the later one.
+ Merge "Artist Role" with "Artist Prefix"


In [3]:
## modify the names of columns for indexing convenience
collection_df.columns = ['_'.join(col_name.split()) for col_name in collection_df]

In [16]:
## Check "Object Begin Date" and "Object End Date" with "Object Date"
def check_Object_Date(row):
    return row.Object_End_Date >= row.Object_Begin_Date

def modify_Object_Date(df):
    test_df = df[df.Object_Date.notna()]
    
    for i in test_df.index:
        obdate = df.Object_Date.loc[i]
        year = re.compile("\d+").findall(obdate)
        assert len(year) > 0
        if "-" in obdate:
            if 'B.C.' in obdate:
                bdate = -int(year[0])*100
                edate = -int(year[1])*100
            elif 'century' in obdate:
                bdate = int(year[0])*100
                edate = int(year[1])*100
            elif len(year[0]) == 4 and len(year[1]) == 4:
                bdate = int(year[0])
                edate = int(year[1])
            elif len(year[0]) == 4 and len(year[1]) == 2:
                bdate = int(year[0])
                edate = bdate//100*100 + int(year[1])
            else:
                print('case1',df.loc[i,['Object_Date','Object_Begin_Date','Object_End_Date']],sep = '\n')
                break
        else:
            if 'B.C.' in obdate:
                bdate = -int(year[0])*100 + 1
                edate = bdate + 99
            elif 'century' in obdate:
                bdate = (int(year[0])-1)*100
                edate = bdate + 99
            elif len(year[0]) == 4:
                bdate = edate = int(year[0])
            else:
                print('case2',df.loc[i,['Object_Date','Object_Begin_Date','Object_End_Date']],sep = '\n')
                break
        if df.loc[i].Object_Begin_Date < bdate:
            df.loc[i].Object_Begin_Date = bdate
        if df.loc[i].Object_End_Date > edate:
            df.loc[i].Object_End_Date = edate
    return df

#     a = test_df[ ["century" in date for date in test_df.Object_Date]][['Object_Date','Object_Begin_Date','Object_End_Date']]
#     return a[-a.apply(check_Object_Date,axis = 1)][['Object_Date','Object_Begin_Date','Object_End_Date']]

modify_Object_Date(collection_df)

/home/cesare/.local/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


IndexError: list index out of range

In [20]:
## Check "Object Begin Date" and "Object End Date" with "Object Date"
def check_Object_Date(row):
    return row.Object_End_Date < row.Object_Begin_Date

def modify_Object_Date(df):
    
    return df[df.apply(check_Object_Date,axis = 1)]

modify_Object_Date(collection_df)[['Object_Date','Object_Begin_Date','Object_End_Date']]

,Object_Date,Object_Begin_Date,Object_End_Date
27396,5th–1st century B.C.,5,0
33099,ca. 50 B.C.,-45,-55
47779,"1861, 9th month",1861,9
80380,1800–1875,1875,1800
120402,late 18th century,1785,1779
138066,late 18th century,1785,1773
140139,late 18th century,1785,1779
144678,7th–early 5th century B.C.,0,-5
148312,4th century B.C.,-399,-3000
150032,ca. 400 B.C.,-450,-475


In [17]:
test_df = collection_df[collection_df.Object_Date.notna()]
test_df[["-" in i and len(re.compile("\d+").findall(i)) < 2 for i in test_df.Object_Date]][['Object_Date','Object_Begin_Date','Object_End_Date']]

,Object_Date,Object_Begin_Date,Object_End_Date
11056,mid-1880s,1880,1880
12656,mid-18th century,1757,1760
13068,mid-19th century,1825,1875
14120,mid-19th century,1825,1875
16164,mid-18th century,1740,1760
16165,mid-18th century,1740,1760
16166,mid-18th century,1740,1760
16167,mid-18th century,1740,1760
16168,mid-18th century,1740,1760
16184,mid-18th century,1740,1760


We intend to check number of NaNs in all the columns first and try to ingnore those columns which the number of NaNs is no more than 10%.

In [4]:
collection_df.isna().sum()

Object_Number                   0
Is_Highlight                    0
Is_Public_Domain                0
Object_ID                       0
Department                      0
Object_Name                  4250
Title                       31083
Culture                    282791
Period                     402255
Dynasty                    468361
Reign                      480444
Portfolio                  469642
Artist_Role                208360
Artist_Prefix              393612
Artist_Display_Name        206270
Artist_Display_Bio         254730
Artist_Suffix              479982
Artist_Alpha_Sort          206303
Artist_Nationality         298188
Artist_Begin_Date          252578
Artist_End_Date            255352
Object_Date                 15037
Object_Begin_Date               0
Object_End_Date                 0
Medium                       7669
Dimensions                  76663
Credit_Line                   706
Geography_Type             431379
City                       459489
State         

In [5]:
collection_df.dtypes

Object_Number              object
Is_Highlight                 bool
Is_Public_Domain             bool
Object_ID                   int64
Department                 object
Object_Name                object
Title                      object
Culture                    object
Period                     object
Dynasty                    object
Reign                      object
Portfolio                  object
Artist_Role                object
Artist_Prefix              object
Artist_Display_Name        object
Artist_Display_Bio         object
Artist_Suffix              object
Artist_Alpha_Sort          object
Artist_Nationality         object
Artist_Begin_Date          object
Artist_End_Date            object
Object_Date                object
Object_Begin_Date           int64
Object_End_Date             int64
Medium                     object
Dimensions                 object
Credit_Line                object
Geography_Type             object
City                       object
State         

In [6]:
filtered_df =  collection_df.loc[:, collection_df.isna().sum(axis = 0) < len(collection_df.index)*0.1 ]
filtered_df

,Object_Number,Is_Highlight,Is_Public_Domain,Object_ID,Department,Object_Name,Title,Object_Date,Object_Begin_Date,Object_End_Date,Medium,Credit_Line,Link_Resource,Metadata_Date,Repository
0,1979.486.1,False,False,1,American Decorative Arts,Coin,One-dollar Liberty Head Coin,1853,1853,1853,Gold,"Gift of Heinz L. Stoppelmann, 1979",http://www.metmuseum.org/art/collection/search/1,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
1,1980.264.5,False,False,2,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,1901,1901,1901,Gold,"Gift of Heinz L. Stoppelmann, 1980",http://www.metmuseum.org/art/collection/search/2,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
2,67.265.9,False,False,3,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/3,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
3,67.265.10,False,False,4,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/4,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
4,67.265.11,False,False,5,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/5,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
5,67.265.12,False,False,6,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/6,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
6,67.265.13,False,False,7,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/7,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
7,67.265.14,False,False,8,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/8,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
8,67.265.15,False,False,9,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,1909–27,1909,1927,Gold,"Gift of C. Ruxton Love, Jr., 1967",http://www.metmuseum.org/art/collection/search/9,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"
9,1979.486.3,False,False,10,American Decorative Arts,Coin,Two-and-a-half-dollar Indian Head Coin,1912,1912,1912,Gold,"Gift of Heinz L. Stoppelmann, 1979",http://www.metmuseum.org/art/collection/search/10,2/25/2019 8:00:02 AM,"Metropolitan Museum of Art, New York, NY"


## Data analysis & Data Visualization

In [12]:
## explore the type of object
objects_amount = collection_df.Object_Name.value_counts(dropna=False)
objects_amount.head(10)

Print                   101206
Photograph               29594
Drawing                  25508
Book                     13495
Fragment                  9667
Kylix fragment            8927
Piece                     8632
Painting                  6120
Negative                  5940
Baseball card, print      5543
Name: Object_Name, dtype: int64

In [17]:
## explore the departement
departments = collection_df.Department.value_counts()
departments

Drawings and Prints                          176173
European Sculpture and Decorative Arts        42975
Photographs                                   39042
Asian Art                                     37460
Greek and Roman Art                           33698
Costume Institute                             31402
Egyptian Art                                  27912
American Decorative Arts                      18566
Islamic Art                                   15784
Modern and Contemporary Art                   14806
Arms and Armor                                13575
Arts of Africa, Oceania, and the Americas     13053
Medieval Art                                   7503
Ancient Near Eastern Art                       6188
Musical Instruments                            5321
European Paintings                             2942
The Cloisters                                  2629
Robert Lehman Collection                       2586
The Libraries                                   252
Name: Depart

## Analysis conclusion